# Advanced Lane Finding Project

The goals / steps of this project are the following:

- [x] Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
- [x] Apply a distortion correction to raw images.
- [x] Use color transforms, gradients, etc., to create a thresholded binary image.
- [ ] Apply a perspective transform to rectify binary image ("birds-eye view").
- [ ] Detect lane pixels and fit to find the lane boundary.
- [ ] Determine the curvature of the lane and vehicle position with respect to center.
- [ ] Warp the detected lane boundaries back onto the original image.
- [ ] Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Imports

In [ ]:
# builtins
from glob import glob
import functools
import os

# third-party
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# project-specific
from src.calibrate import calibrate
from src.thresholding import region_of_interest, thresh, and_binary, or_binary
from src.image_mappings import direction, magnitude

## Calibration

In [ ]:
print("Finding all calibration images...")
calibration_image_paths = sorted(glob('./camera_cal/calibration*.jpg'), key=lambda p: (len(p), p))
calibration_images = list(map(cv2.imread, calibration_image_paths))
for path, image in zip(calibration_image_paths, calibration_images):
    print("{0} ({1[0]}w, {1[1]}h)".format(
        os.path.basename(path),
        image.shape
    ))
print("Found {} images.".format(len(calibration_images)))

print("Calibrating...")
used_calibration_images, object_points, all_corners, undistort, mtx, dist = calibrate(
    calibration_images, 
    chessboard_dimensions=(9, 6), 
    to_grayscale_flag=cv2.COLOR_BGR2GRAY
)
print("Calibrated using {} chessboard images (was unable to use {} images).".format(
    len(used_calibration_images), len(calibration_images) - len(used_calibration_images)
))

print("Writing to output_images/chessboards...")
directory = "output_images/chessboards"
try:
    os.mkdir(directory)
except:
    pass

for image, corners, i in zip(used_calibration_images, all_corners, range(len(used_calibration_images))):
    img = cv2.drawChessboardCorners(image, (9, 6), corners, True)
    cv2.imwrite(directory + "/{}.before.jpg".format(i), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    out = undistort(img)
    cv2.imwrite(directory + "/{}.after.jpg".format(i), cv2.cvtColor(out, cv2.COLOR_RGB2BGR))
    if i == 0:
        plt.imshow(out)

print("Written. Sample output with corners drawn:")

## Thresholding

In [ ]:
print("Defining image selectors...")
# Yes. Lots of fiddling with numbers happened here. Can't wait for ML approach.
    
def asphalt_selector(img, convert_to_hls_flag=cv2.COLOR_BGR2HLS, convert_to_gray_flag=cv2.COLOR_BGR2GRAY):
    hls = cv2.cvtColor(img, convert_to_hls_flag)
    gray = cv2.cvtColor(img, convert_to_gray_flag)
    return and_binary(
        thresh(direction(
            thresh(
                hls[:,:,1], 
                (140, 240)
            ), 
            sobel_kernel=5
        ), (0.7, 1.2)),
        # &
        thresh(magnitude(
            gray, 
            sobel_kernel=13
        ), (100, 256)),
    )

def concrete_selector(img, convert_to_hls_flag=cv2.COLOR_BGR2HLS, convert_to_gray_flag=cv2.COLOR_BGR2GRAY):
    hls = cv2.cvtColor(img, convert_to_hls_flag)
    return thresh(magnitude(
            thresh(direction(
                thresh(hls[:,:,2], (125, 256)),
                sobel_kernel=5,
                ), (0.7, 1.2),
            ), 
            sobel_kernel=13,
        ), (110, 256),
    )


def lane_selector(img, convert_to_hls_flag=cv2.COLOR_BGR2HLS, convert_to_gray_flag=cv2.COLOR_BGR2GRAY):
    return or_binary(
        asphalt_selector(img, convert_to_hls_flag=convert_to_hls_flag, convert_to_gray_flag=convert_to_gray_flag),
        concrete_selector(img, convert_to_hls_flag=convert_to_hls_flag, convert_to_gray_flag=convert_to_gray_flag),
    )

def region_selector(i):
    return region_of_interest(
        i, 
        np.array([[
            ((i.shape[1] * 1) / 16, i.shape[0]),
            ((i.shape[1] * 7) / 16, i.shape[0] * .61),
            ((i.shape[1] * 9) / 16, i.shape[0] * .61),
            ((i.shape[1] * 15) / 16, i.shape[0])
        ]], dtype=np.int32),
    )

print("Defined.")


In [ ]:
img = cv2.imread("test_images/straight_lines2.jpg")
rgb = lambda bgr: cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

asphalt = asphalt_selector(img)
concrete = concrete_selector(img)
lanes = lane_selector(img)

f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
ax1.imshow(rgb(img))
ax2.imshow(asphalt, cmap='gray')
ax3.imshow(concrete, cmap='gray')
ax4.imshow(lanes, cmap='gray')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
print("Processing sample images...")

for image_name in sorted(glob("test_images/*.jpg"), key=lambda p: (len(p), p)):
    img = cv2.imread(image_name)
    directory = "output_images/" + os.path.basename(image_name)
    try:
        os.mkdir(directory)
    except:
        pass
    print(directory)

    cv2.imwrite(directory + "/original.jpg", img)
    
    # Undistort image (virtue of camera lens)
    undis = undistort(img)
    cv2.imwrite(directory + "/undistorted.jpg", undis)
    
    # Select lanes
    asphalt = asphalt_selector(undis)  # for show
    plt.imsave(directory + "/asphalt_selector.jpg", asphalt, cmap='gray')
    concrete = concrete_selector(undis)  # for show
    plt.imsave(directory + "/concrete_selector.jpg", concrete, cmap='gray')

    lanes = lane_selector(undis)
    plt.imsave(directory + "/lane_selector.jpg", lanes, cmap='gray')

    region = region_selector(lanes)
    plt.imsave(directory + "/region_selector.jpg", region, cmap='gray')

print("Done.")
